In [3]:
import pandas as pd
import os

In [242]:
df_catch = pd.read_csv(os.path.join('db1','catch.csv'))

In [265]:
df_product = pd.read_csv(os.path.join('db1','product.csv'))

In [ ]:
df_ext = pd.read_csv(os.path.join('db2','Ext.csv'))
df_ext2 = pd.read_csv(os.path.join('db2','Ext2.csv'))

## Preproccessing

#### Catch

In [263]:
df_catch.head()

,id_ves,date,id_region,id_fish,catch_volume,id_regime,permit,id_own
0,1628,2022-01-01,272,400,31.874,41,2824,961
1,70,2022-01-01,277,886,0.476,1,2414,1281
2,1608,2022-01-01,116,409,26.310,15,3031,1227
3,832,2022-01-01,272,292,9.541,1,1894,1281
4,832,2022-01-01,272,113,0.062,1,1894,1281


Drop unused `permit`

In [273]:
df_db1 = df_catch.drop(columns='permit')

### Ext
Check how many id_ves from df_catch contains in df_ext

In [262]:
df_ext[~(df_ext['id_ves'].isin(df_catch['id_ves'].unique())) & (df_ext['id_ves']!=-1)]

,id_ves,id_own,date_fishery,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
44,1579,1195,2022-05-24 00:00:00,6289005,6672408,\N,\N,\N
45,1579,1195,2022-05-24 00:00:00,6289005,8469633,\N,\N,\N
46,1579,1195,2022-05-24 00:00:00,6289005,8605162,\N,\N,\N
47,1579,1195,2022-05-24 00:00:00,6289005,7798565,\N,\N,\N
48,1599,1195,2022-05-24 00:00:00,6290078,7952695,\N,\N,\N
...,...,...,...,...,...,...,...,...
3260721,1636,1144,2020-12-31 00:00:00,6290871,7971576,\N,\N,\N
3260730,1466,938,2020-12-31 00:00:00,6295910,8909315,\N,\N,\N
3260731,1466,938,2020-12-31 00:00:00,6295910,8375262,\N,\N,\N
3260742,1363,1011,2020-12-31 00:00:00,6292142,6553285,\N,\N,\N


It's many. This why make different DBSCAN for `df_catch` only, `df_ext` only and compare `df_catch` and `df_ext`

#### Remove duplicates

In [241]:
df_ext[df_ext['id_ves']==-1]

,id_ves,id_own,date_fishery,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
15313,-1,-1,2022-04-20 02:38:01,6294403,6834025,9518351,\N,Камчатский край
15314,-1,-1,2022-04-20 02:38:01,6294403,8596508,9518351,\N,Камчатский край
15315,-1,-1,2022-04-20 02:37:57,6295967,9079011,9516052,\N,Приморский край
15316,-1,-1,2022-04-20 02:37:34,6295967,8489049,9516052,\N,Приморский край
15317,-1,-1,2022-04-20 02:37:04,6295967,8183544,9516052,\N,Приморский край
...,...,...,...,...,...,...,...,...
3112990,-1,-1,2022-01-02 00:02:35,6294690,7601012,9515018,2022-01-02 00:00:00,г. Москва
3112991,-1,-1,2022-01-02 00:02:35,6294690,8450974,9515018,2022-01-02 00:00:00,г. Москва
3112992,-1,-1,2022-01-02 00:02:35,6294690,6391908,9515018,2022-01-02 00:00:00,г. Москва
3112993,-1,-1,2022-01-02 00:02:19,6294190,7006117,9513603,2022-01-02 00:00:00,Республика Хакасия


In [168]:
df_ext.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3260798 entries, 0 to 3260797
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   id_ves          int64 
 1   id_own          int64 
 2   date_fishery    object
 3   numPart         object
 4   id_Plat         int64 
 5   id_vsd          int64 
 6   Name_Plat       object
 7   Product_period  object
 8   Region_Plat     object
dtypes: int64(4), object(5)
memory usage: 223.9+ MB


There just 4 unique values. Let's see them

In [170]:
df_ext[df_ext['numPart']!='\\N']

,id_ves,id_own,date_fishery,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
3154783,1290,1156,2021-09-14 00:00:00,1,6296848,6760283,\N,\N,\N
3154784,1290,1156,2021-09-14 00:00:00,1,6296848,8508555,\N,\N,\N
3154785,1290,1156,2021-09-14 00:00:00,3,6296848,9447157,\N,\N,\N
3154786,1290,1156,2021-09-14 00:00:00,2,6296848,7956670,\N,\N,\N
3154787,1290,1156,2021-09-14 00:00:00,3,6296848,8746726,\N,\N,\N
3154788,1322,1023,2021-09-14 00:00:00,3,6294344,6319541,\N,\N,\N
3154789,1322,1023,2021-09-14 00:00:00,1,6294344,8957890,\N,\N,\N
3154790,1322,1023,2021-09-14 00:00:00,3,6294344,9167480,\N,\N,\N
3154791,1322,1023,2021-09-14 00:00:00,3,6294344,8054779,\N,\N,\N
3154792,1322,1023,2021-09-14 00:00:00,2,6294344,9329734,\N,\N,\N


Not intresting. Remove this feature.

In [171]:
df_ext.drop(columns=['numPart'], inplace=True)
df_ext[df_ext['id_vsd'].duplicated()].head(3)

,id_ves,id_own,date_fishery,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
54815,1620,996,2022-04-19 00:00:00,6290172,6748647,\N,\N,\N
54816,1657,1175,2022-04-19 00:00:00,6292428,6753918,\N,\N,\N
54817,1531,1011,2022-04-19 00:00:00,6293278,8559213,\N,\N,\N


In [172]:
def count_duplicated(df,id):
    return len(df[df[id].duplicated()])

'%s record is duplicated' % count_duplicated(df_ext,'id_vsd')

'45854 record is duplicated'

We see duplicated values, check what is duplicated records

In [173]:
df_ext[df_ext['id_vsd']==9232550]

,id_ves,id_own,date_fishery,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
1312329,-1,-1,2022-03-07 03:44:45,6289288,9232550,9514518,2022-03-01 00:00:00,Красноярский край


We see that it is same records but with more information. Try remerge same records in one.

In [174]:
df_ext_descr = df_ext[(df_ext['Name_Plat'] != '\\N') | (df_ext['Product_period'] != '\\N') | (df_ext['Region_Plat'] != '\\N')]

Check that in description table not duplicated values

In [175]:
print('In decription table %s records is duplicated' % count_duplicated(df_ext_descr,'id_vsd'))

In decription table 0 records is duplicated


In [176]:
'Unique %s ids' % len(df_ext['id_vsd'].unique())

'Unique 3214944 ids'

Merge tables

In [177]:
df_ext_wo_data=df_ext.drop(df_ext_descr.index)

In [178]:
df_ext_merged = pd.merge(
    pd.DataFrame(df_ext['id_vsd'].unique(), columns=['id_vsd']),
    df_ext_wo_data[['id_vsd','id_own','id_ves','date_fishery','id_Plat']],
    on='id_vsd',
    how="left"
)
len(df_ext_merged)

3214944

In [179]:
df_ext_merged = pd.merge(
    df_ext_merged,
    df_ext_descr[['id_vsd','date_fishery','Name_Plat','Product_period','Region_Plat']],
    on='id_vsd',
    how="left"
)
len(df_ext_merged)

3214944

Fill `date_fishery` from describted table

In [180]:
df_ext_merged.loc[~df_ext_merged['date_fishery_y'].isna(),'date_fishery_x']=df_ext_merged['date_fishery_y']
df_ext_merged.drop(columns='date_fishery_y',inplace=True)
len(df_ext_merged)

3214944

Same work with df_ext2

In [181]:
df_ext2[df_ext2['id_vsd'].duplicated()].head(3)

,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit
556258,6334548,5445449,-1,пикша свежая,2022-03-21 02:54:35,3,тонна
556259,6306527,5662445,-1,треска,2022-03-21 02:54:34,1,тонна
556260,7080120,5559522,-1,окунь-клювач,2022-03-21 02:54:33,0,тонна


In [182]:
'%s record is duplicated' % count_duplicated(df_ext2,'id_vsd')

'45854 record is duplicated'

In [183]:
df_ext2[df_ext2['id_vsd']==5544870]

,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit


Seems that in duplicated records contains round values. We get more descridable records

In [184]:
'Unique %s ids' % len(df_ext2['id_vsd'].unique())

'Unique 3069426 ids'

In [185]:
df_ext2_w_fish_id = df_ext2[df_ext2['id_fish']!=-1]
print('In table with fish_id %s records is duplicated' % count_duplicated(df_ext2_w_fish_id,'id_vsd'))
df_ext2_wo_fish_id = df_ext2.drop(df_ext2_w_fish_id.index)
print('In table without fish_id %s records is duplicated' % count_duplicated(df_ext2_w_fish_id,'id_vsd'))

In table with fish_id 0 records is duplicated
In table without fish_id 0 records is duplicated


In [186]:
df_ext2_wo_fish_id.head()

,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit
39891,8753597,3981889,-1,скумбрия холодного копчения,2022-01-14 09:25:25,250,кг
39892,9199246,4568206,-1,икра сельди соленая,2022-01-04 05:16:29,3,кг
39893,8394699,5849836,-1,готовые блюда из рыбы,2022-02-03 15:43:10,2,кг
39894,8375000,6074372,-1,путассу солено-сушеная,2022-03-21 08:49:01,174,кг
39895,9307607,3322017,-1,путассу вяленая,2022-03-21 06:22:35,50,кг


In [187]:
df_ext2_merged = pd.merge(
    pd.DataFrame(df_ext2['id_vsd'].unique(), columns=['id_vsd']),
    df_ext2_w_fish_id[['id_vsd','id_fish','fish','volume','unit','date_vsd']],
    on='id_vsd',
    how="left"
)
len(df_ext2_merged)

3069426

In [188]:
df_ext2_merged = pd.merge(
    df_ext2_merged,
    df_ext2_wo_fish_id[['id_vsd','fish','date_vsd','volume','unit']],
    on='id_vsd',
    how="left"
)
len(df_ext2_merged)

3069426

In [189]:
print('In table without fish_id %s records is duplicated' % count_duplicated(df_ext2_w_fish_id,'id_vsd'))

In table without fish_id 0 records is duplicated


In [190]:
df_ext2_merged[~(df_ext2_merged['volume_x'].isna())].head(3)

,id_vsd,id_fish,fish_x,volume_x,unit_x,date_vsd_x,fish_y,date_vsd_y,volume_y,unit_y
0,7820377,400.0,минтай,75480.0,\N,2022-01-01 00:39:56,NaN,NaN,NaN,NaN
1,9397385,400.0,минтай,62586.0,\N,2022-01-01 01:34:08,NaN,NaN,NaN,NaN
2,8666689,292.0,треска,1155.0,\N,2022-01-01 01:34:11,NaN,NaN,NaN,NaN


Fill na from describe dataset, drop unneccesary columns and fillna -1

In [191]:
if_fish_na = df_ext2_merged['id_fish'].isna()
df_ext2_merged.loc[if_fish_na,'fish_x']=df_ext2_merged['fish_y']
df_ext2_merged.loc[if_fish_na,'volume_x']=df_ext2_merged['volume_y']
df_ext2_merged.loc[if_fish_na,'unit_x']=df_ext2_merged['unit_y']
df_ext2_merged.loc[if_fish_na,'date_vsd_x']=df_ext2_merged['date_vsd_y']
df_ext2_merged=df_ext2_merged.drop(columns=['fish_y','volume_y','unit_y','date_vsd_y'])
len(df_ext2_merged)

3069426

Now try normalize fish and volume

In [192]:
df_ext2_merged['unit_x'].unique()

array(['\\N', 'кг', 'тонна'], dtype=object)

In [193]:
df_ext2_merged.loc[df_ext2_merged['unit_x']=='\\N','volume_x']/=1000
df_ext2_merged.loc[df_ext2_merged['unit_x']=='кг','volume_x']/=1000

In [194]:
fish_class = df_ext2_merged[df_ext2_merged['id_fish']>=0][['id_fish','fish_x']].drop_duplicates()
df_ext2_merged=pd.merge(
    df_ext2_merged,
    fish_class,
    on='fish_x',
    how='left'
)
df_ext2_merged.loc[df_ext2_merged['id_fish_x'].isna(),'id_fish_x']=df_ext2_merged['id_fish_y']
df_ext2_merged.drop(columns='id_fish_y', inplace=True)

In [195]:
top_fish=df_ext2_merged.groupby('fish_x').sum().sort_values('volume_x', ascending=False)

Remove records with zero volume

In [196]:
df_ext2_merged=df_ext2_merged[~(df_ext2_merged['fish_x'].isin(top_fish[top_fish['volume_x']==0].index))]

In [197]:
top_fish[:10]
# todo classify fish by name

,id_vsd,id_fish_x,volume_x
fish_x,,,
минтай,442513491360,22391600.0,985755865.0
треска,464203028150,13024022.0,449742126.0
минтай б/г мороженый,62593218774,0.0,281485109.0
сельдь тихоокеанская,11995594961,308856.0,179568763.0
сельдь тихоокеанская н/р мороженая,9425736835,0.0,157704270.0
путассу (северная),4787364269,246627.0,85655049.0
филе минтая мороженое,72158586141,0.0,85229755.0
пикша мороженая,5332610003,0.0,52233340.0
минтай н/р мороженый,8716334919,0.0,50878872.0


Merge and dump db2

In [198]:
df_db2 = pd.merge(df_ext_merged,df_ext2_merged,on='id_vsd')
df_db2.to_csv('db2.csv',index=None)

In [280]:
df_db2=pd.read_csv('db2.csv')